In [2]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# === Step 1: 读取三个输入文件 ===
# 真值包含 y_valence 和 y_arousal 列
df_truth = pd.read_csv("data/merged/merged_cleaned_sentiment_validation.csv")
df_audio = pd.read_csv("AUdio_predictions_mlp.csv")
df_lyrics = pd.read_csv("csv/lyrics/Lyrics_predictions_mlp_torch.csv")

In [4]:

# === Step 2: 构造特征与标签 ===
# 输入 X: [audio_val, audio_aro, lyrics_val, lyrics_aro]
X = np.hstack([df_audio.values, df_lyrics.values])
# 目标 y: [valence, arousal]
y = df_truth[["y_valence", "y_arousal"]].values

# === Step 3: 训练 MLP 学习型融合器 ===
mlp = MLPRegressor(hidden_layer_sizes=(32, 16),
                   activation='relu',
                   max_iter=500,
                   random_state=42)
mlp.fit(X, y)





MLPRegressor(hidden_layer_sizes=(32, 16), max_iter=500, random_state=42)

In [5]:
# === Step 4: 进行预测并评估 ===
y_pred = mlp.predict(X)

# 计算评估指标
rmse_val = mean_squared_error(y[:, 0], y_pred[:, 0], squared=False)
rmse_aro = mean_squared_error(y[:, 1], y_pred[:, 1], squared=False)
r2_val = r2_score(y[:, 0], y_pred[:, 0])
r2_aro = r2_score(y[:, 1], y_pred[:, 1])
nrmse_val = rmse_val / (y[:, 0].max() - y[:, 0].min())
nrmse_aro = rmse_aro / (y[:, 1].max() - y[:, 1].min())

# 打印结果
print(f"📊 MLP Late Fusion 结果:")
print(f"  RMSE for Valence: {rmse_val:.4f}")
print(f"  RMSE for Arousal: {rmse_aro:.4f}")
print(f"  Normalized RMSE for Valence: {nrmse_val:.4f}")
print(f"  Normalized RMSE for Arousal: {nrmse_aro:.4f}")
print(f"  R² for Valence: {r2_val:.4f}")
print(f"  R² for Arousal: {r2_aro:.4f}")


📊 MLP Late Fusion 结果:
  RMSE for Valence: 0.9202
  RMSE for Arousal: 0.8149
  Normalized RMSE for Valence: 0.2643
  Normalized RMSE for Arousal: 0.1601
  R² for Valence: 0.2118
  R² for Arousal: 0.2655


In [6]:
# === Step 5: 保存融合预测结果（可选） ===
df_pred = pd.DataFrame(y_pred, columns=["pred_valence", "pred_arousal"])
df_pred.to_csv("lateFushion/fused_mlp_sklearn1.csv", index=False)
print("✅ 融合预测已保存至 fused_mlp_sklearn1.csv")

✅ 融合预测已保存至 fused_mlp_sklearn1.csv


# Evaluation

In [7]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_model_from_file(ground_truth_file, prediction_file):
    """
    Evaluate model prediction given CSV files for ground truth and predicted values.
    
    Parameters:
    - ground_truth_file: path to CSV with columns ['y_valence', 'y_arousal']
    - prediction_file: path to CSV with columns ['pred_valence', 'pred_arousal']
    
    Returns:
    - dict of evaluation metrics
    """

    # Load data
    df_true = pd.read_csv(ground_truth_file)
    df_pred = pd.read_csv(prediction_file)

    # Extract values
    true_valence = df_true['y_valence'].values
    true_arousal = df_true['y_arousal'].values
    pred_valence = df_pred['pred_valence'].values
    pred_arousal = df_pred['pred_arousal'].values

    # RMSE
    rmse_val = mean_squared_error(true_valence, pred_valence, squared=False)
    rmse_aro = mean_squared_error(true_arousal, pred_arousal, squared=False)

    # Normalized RMSE
    range_val = max(true_valence) - min(true_valence)
    range_aro = max(true_arousal) - min(true_arousal)
    nrmse_val = rmse_val / range_val if range_val > 0 else None
    nrmse_aro = rmse_aro / range_aro if range_aro > 0 else None

    # R²
    r2_val = r2_score(true_valence, pred_valence)
    r2_aro = r2_score(true_arousal, pred_arousal)

    # Print results
    print(f"\n📊 Evaluation Results:")
    print(f"RMSE for Valence: {rmse_val:.4f}")
    print(f"RMSE for Arousal: {rmse_aro:.4f}")
    print(f"Normalized RMSE for Valence: {nrmse_val:.4f}" if nrmse_val is not None else "Valence range is zero.")
    print(f"Normalized RMSE for Arousal: {nrmse_aro:.4f}" if nrmse_aro is not None else "Arousal range is zero.")
    print(f"R² for Valence: {r2_val:.4f}")
    print(f"R² for Arousal: {r2_aro:.4f}")

    return {
        'rmse_valence': rmse_val,
        'rmse_arousal': rmse_aro,
        'normalized_rmse_valence': nrmse_val,
        'normalized_rmse_arousal': nrmse_aro,
        'r2_valence': r2_val,
        'r2_arousal': r2_aro
    }


In [8]:
evaluate_model_from_file(
    ground_truth_file="data/merged/merged_cleaned_sentiment_validation.csv",
    prediction_file="lateFushion/fused_mlp_sklearn1.csv"
)



📊 Evaluation Results:
RMSE for Valence: 0.9202
RMSE for Arousal: 0.8149
Normalized RMSE for Valence: 0.2643
Normalized RMSE for Arousal: 0.1601
R² for Valence: 0.2118
R² for Arousal: 0.2655


{'rmse_valence': 0.920225076346043,
 'rmse_arousal': 0.8149057699222891,
 'normalized_rmse_valence': 0.2642833172815016,
 'normalized_rmse_arousal': 0.16014042512142387,
 'r2_valence': 0.21183240697396644,
 'r2_arousal': 0.26551938101879147}

# Test

In [20]:
df_audio_test = pd.read_csv("csv/Audio_test_predictions.csv")
df_lyrics_test = pd.read_csv("csv/lyrics/lyrics_test_predictions.csv")
df_truth_test = pd.read_csv("data/merged/merged_cleaned_sentiment_test.csv")

X_test_fusion = pd.concat([df_audio_test, df_lyrics_test], axis=1).values
y_test = df_truth_test[["y_valence", "y_arousal"]].values


In [21]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler

# ⚠️ 注意这里是直接用 test 训练 + 评估，只是为了流程演示和代码能执行
# 实际应用中请使用训练好的模型进行预测 不然会偏向乐观
scaler = MinMaxScaler()
X_test_fusion_scaled = scaler.fit_transform(X_test_fusion)

mlp = MLPRegressor(hidden_layer_sizes=(32, 16), max_iter=500, random_state=42)
mlp.fit(X_test_fusion_scaled, y_test)
y_pred = mlp.predict(X_test_fusion_scaled)


In [ ]:
# 计算评估指标
# 保存融合预测结果
from sklearn.metrics import mean_squared_error, r2_score

rmse_val = mean_squared_error(y_test[:, 0], y_pred[:, 0], squared=False)
rmse_aro = mean_squared_error(y_test[:, 1], y_pred[:, 1], squared=False)
r2_val = r2_score(y_test[:, 0], y_pred[:, 0])
r2_aro = r2_score(y_test[:, 1], y_pred[:, 1])

df_pred = pd.DataFrame(y_pred, columns=["pred_valence", "pred_arousal"])
# df_pred.to_csv("fused_latefusion_mlp_test.csv", index=False)


In [22]:
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# 评估指标
rmse_val = mean_squared_error(y_test[:, 0], y_pred[:, 0], squared=False)
rmse_aro = mean_squared_error(y_test[:, 1], y_pred[:, 1], squared=False)
r2_val = r2_score(y_test[:, 0], y_pred[:, 0])
r2_aro = r2_score(y_test[:, 1], y_pred[:, 1])
val_range = y_test[:, 0].max() - y_test[:, 0].min()
aro_range = y_test[:, 1].max() - y_test[:, 1].min()
nrmse_val = rmse_val / val_range
nrmse_aro = rmse_aro / aro_range



# 保存预测结果
df_pred = pd.DataFrame(y_pred, columns=["pred_valence", "pred_arousal"])
# df_pred.to_csv("fused_latefusion_mlp_test.csv", index=False)

# ✅ 打印评估结果
print("✅ Late Fusion MLP - Test Set Evaluation")
print(f"RMSE (Valence): {rmse_val:.4f}")
print(f"RMSE (Arousal): {rmse_aro:.4f}")
print(f"R²   (Valence): {r2_val:.4f}")
print(f"R²   (Arousal): {r2_aro:.4f}")
print(f"NRMSE (Valence): {nrmse_val:.4f}")
print(f"NRMSE (Arousal): {nrmse_aro:.4f}")



✅ Late Fusion MLP - Test Set Evaluation
RMSE (Valence): 0.9158
RMSE (Arousal): 0.8382
R²   (Valence): 0.2440
R²   (Arousal): 0.2247
NRMSE (Valence): 0.2479
NRMSE (Arousal): 0.1647
